## Usando esparcidade para regularizar modelos

O método LASSO (least absolute shrinkage and selection operator - operador de seleção e retração mínima absoluta) é muito similar a regressão de cuumeira e ao LARS (least angle regression - regressão de menor ângulo)

## Como fazer

1. utilizar a função `make_regression` e criar um conjunto de dados 

In [1]:
import numpy as np
from sklearn.datasets import make_regression
reg_data, reg_target = make_regression(n_samples=200, n_features=500, n_informative=5, noise=5)

2.  importar o objeto LASSO

In [3]:
from sklearn.linear_model import Lasso
lasso = Lasso()

3. Lasso contêm muito parâmetros, mas um dos mais interssantes é o alfa (alpha), este dimensiona o termo de penalização do método lasso. Agora vai ficar com 1, e se é igual a zero, vira uma regressão linear. 

In [4]:
lasso.fit(reg_data, reg_target)

Lasso()

4. ver quantos dos coeficientes permanecer diferente de zero

In [5]:
np.sum(lasso.coef_ != 0)

11

Colocando zero como parâmetro na função `Lasso()`, vira uma regressão linear, e o próprio scikit recomenda usar regressão linear

In [6]:
lasso_0 = Lasso(0)
lasso_0.fit(reg_data, reg_target)

/home/io/anaconda3/envs/notebook/lib/python3.12/site-packages/sklearn/base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/home/io/anaconda3/envs/notebook/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0)

In [7]:
np.sum(lasso_0.coef_ != 0)

500

## Como funciona

### Lasso CV

Escolhe o lambda(alpha) apropriado é o problema mais crítico.

In [8]:
from sklearn.linear_model import LassoCV

In [9]:
lassocv = LassoCV()

In [10]:
lassocv.fit(reg_data, reg_target)

LassoCV()

In [11]:
lassocv.alpha_

0.818755157354021

Para acessar ao coeficientes, é da seguinte forma

In [12]:
lassocv.coef_[:5]

array([ 0.,  0.,  0., -0.,  0.])

Permitindo o LassoCV escolher automaticamente o melhor alpha (lambda) deixa com alguns coeficientes diferentes de zero

In [13]:
np.sum(lassocv.coef_ != 0)

19

### Lasso para seleção de feature

Lasso pode ser usado para selecionar **variáveis independentes** para outros métodos, por exemplo pode usar a regresão para obter o número de variáveis apropriadas, para depois usar estas em outro algoritmo..
para isso é preciso criar um a matriz baseada em colunas que não são zero para depois filtrar as colunas diferentes de zero, para escolher as variáveis que queremos

In [14]:
mask = lassocv.coef_ != 0
new_reg_data = reg_data[:,mask]
new_reg_data.shape

(200, 19)

## Uma abordagem fundamental para regularizar com LARS

LARS é uma técnica de regressão para problemas dimensionais.

1. importar os objetos necessários, os dados terão 200 pontos de dados e 500 variáveis, ruído baixo e informação reduzida de caracteristicas.

In [15]:
from sklearn.datasets import make_regression
reg_data, reg_target = make_regression(n_samples=200, n_features=500, n_informative=10, noise=2)

2. Como foi selecionado 10  variáveis informativas, especificar 10 coeficientes diferentes de zero no LARS.

In [16]:
from sklearn.linear_model import Lars
lars = Lars(n_nonzero_coefs=10)
lars.fit(reg_data, reg_target)

Lars(n_nonzero_coefs=10)

In [17]:
np.sum(lars.coef_ != 0)

10

3. porque é mais útil usar menos variáveis, será dividido em 2 partes os dados, para treinar em dois modelos LARS, um com 12 coeficientes diferentes de zero e outro quantidade indeterminada. 

In [18]:
train_n = 100
lars_12 = Lars(n_nonzero_coefs=12)
lars_12.fit(reg_data[:train_n],reg_target[:train_n])
lars_500 = Lars()
lars_500.fit(reg_data[:train_n], reg_target[:train_n])

Lars()

In [19]:
np.mean(np.power(reg_target[train_n:] - lars_12.predict(reg_data[train_n:]),2))

19.212285526061176

4. Para saber como cada carateristicas se adapta para os dados desconhecidos


In [20]:
np.mean(np.power(reg_target[train_n:] - lars_500.predict(reg_data[train_n:]),2))

1.750134741608416e+32

In [21]:
np.sum(lars_500.coef_ != 0)

411

## Mais

Usar validação cruzada (cross-validation) para afinar aa regressão de cresta, pode ser feita do mesmo jeito com LARS

In [22]:
from sklearn.linear_model import LarsCV
lcv = LarsCV()
lcv.fit(reg_data, reg_target)

LarsCV()

utilizando cross-validation  ajudar em determinar o melhor número coeficientes diferentes de zero para uso.

In [23]:
np.sum(lcv.coef_ != 0)

67

In [24]:
lars67 = Lars(n_nonzero_coefs=67)
lars67.fit(reg_data[:train_n], reg_target[:train_n])

Lars(n_nonzero_coefs=67)

In [25]:
np.mean(np.power(reg_target[train_n:] - lars67.predict(reg_data[train_n:]),2))

7.218378417276835

In [26]:
np.sum(lars67.coef_ != 0)

63